In [1]:
import os, sys
from pathlib import Path
import json

nb_dir = Path.cwd()
repo_root = nb_dir.parent
sys.path.insert(0, str(repo_root))

print("Notebook dir:", nb_dir)
print("Repo root:", repo_root)

# %%
import random
import numpy as np
import torch

from src.geo_constraints import DataPaths
from src.dataset_vie import StanfordVIEWellPatchDataset
from src.models.geo_cnn_multitask import GeoCNNMultiTask

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

Notebook dir: h:\GK-MRL-PhysicsConsistent-Inversion\GK-MRL-PhysicsConsistent-Inversion\notebooks
Repo root: h:\GK-MRL-PhysicsConsistent-Inversion\GK-MRL-PhysicsConsistent-Inversion
device: cuda


In [2]:
# %%
DATA_ROOT = r"H:\GK-MRL-PhysicsConsistent-Inversion\GK-MRL-PhysicsConsistent-Inversion\data"
paths = DataPaths(DATA_ROOT)

constraints_npz = os.path.join(paths.processed_dir, "constraints.npz")
assert os.path.isfile(constraints_npz), f"Missing constraints_npz: {constraints_npz}"

ds = StanfordVIEWellPatchDataset(
    paths,
    constraints_npz,
    patch_hw=4,
    use_masked_y=True,
    normalize=True
)

print("processed_dir:", paths.processed_dir)
print("constraints_npz:", constraints_npz)
print("Dataset size:", len(ds))
print("AI mean/std:", ds.ai_mean, ds.ai_std)
print("Seis mean/std:", ds.seis_mean, ds.seis_std)

processed_dir: H:\GK-MRL-PhysicsConsistent-Inversion\GK-MRL-PhysicsConsistent-Inversion\data\processed
constraints_npz: H:\GK-MRL-PhysicsConsistent-Inversion\GK-MRL-PhysicsConsistent-Inversion\data\processed\constraints.npz
Dataset size: 300
AI mean/std: 7.208985805511475 1.2618153095245361
Seis mean/std: 7.208985805511475 1.2618153095245361


In [3]:
# %%
split_dir = os.path.join(paths.processed_dir, "splits")
os.makedirs(split_dir, exist_ok=True)

train_f = os.path.join(split_dir, "train_idx.npy")
val_f   = os.path.join(split_dir, "val_idx.npy")
test_f  = os.path.join(split_dir, "test_idx.npy")

def make_splits(n, seed=2026, frac_train=0.8, frac_val=0.1):
    rng = np.random.default_rng(seed)
    idx = np.arange(n, dtype=np.int32)
    rng.shuffle(idx)
    n_train = int(frac_train * n)
    n_val   = int(frac_val   * n)
    train = idx[:n_train]
    val   = idx[n_train:n_train+n_val]
    test  = idx[n_train+n_val:]
    return train, val, test

if os.path.isfile(train_f) and os.path.isfile(val_f) and os.path.isfile(test_f):
    train_idx = np.load(train_f)
    val_idx   = np.load(val_f)
    test_idx  = np.load(test_f)
    print("Loaded splits:", split_dir)
else:
    train_idx, val_idx, test_idx = make_splits(len(ds), seed=2026)
    np.save(train_f, train_idx)
    np.save(val_f, val_idx)
    np.save(test_f, test_idx)
    print("Created splits:", split_dir)

print("train:", len(train_idx), "val:", len(val_idx), "test:", len(test_idx))

Loaded splits: H:\GK-MRL-PhysicsConsistent-Inversion\GK-MRL-PhysicsConsistent-Inversion\data\processed\splits
train: 210 val: 45 test: 45


In [4]:
# %%
ckpt_dir_final = os.path.join(paths.processed_dir, "checkpoints_multitask_final")
ckpt_joint = os.path.join(ckpt_dir_final, "best_joint.pt")
ckpt_ai    = os.path.join(ckpt_dir_final, "best_ai.pt")

assert os.path.isfile(ckpt_joint), f"Missing: {ckpt_joint}"
assert os.path.isfile(ckpt_ai),    f"Missing: {ckpt_ai}"

print("ckpt_joint:", ckpt_joint)
print("ckpt_ai   :", ckpt_ai)

ckpt_joint: H:\GK-MRL-PhysicsConsistent-Inversion\GK-MRL-PhysicsConsistent-Inversion\data\processed\checkpoints_multitask_final\best_joint.pt
ckpt_ai   : H:\GK-MRL-PhysicsConsistent-Inversion\GK-MRL-PhysicsConsistent-Inversion\data\processed\checkpoints_multitask_final\best_ai.pt


In [5]:
# %%
def build_model():
    # ⚠️这里参数必须与你训练一致：in_channels/base/t/n_facies
    return GeoCNNMultiTask(
        in_channels=7,
        base=32,
        t=200,
        n_facies=4
    ).to(device)

def load_ckpt(model, ckpt_path):
    ckpt = torch.load(ckpt_path, map_location=device)
    state = ckpt["model_state"] if isinstance(ckpt, dict) and "model_state" in ckpt else ckpt
    model.load_state_dict(state, strict=True)
    model.eval()
    return ckpt

m_joint = build_model()
m_ai    = build_model()

_ = load_ckpt(m_joint, ckpt_joint)
_ = load_ckpt(m_ai, ckpt_ai)

print("Loaded best_joint & best_ai.")

Loaded best_joint & best_ai.


C:\Users\Windows\AppData\Local\Temp\ipykernel_35216\1369863937.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(ckpt_path, map_location=device)


In [6]:
# %%
def unpack_outputs(out):
    """
    返回: ai_pred, facies_logits (可能为 None)
    兼容：
      - dict: {"ai":..., "facies":...} / {"ai_pred":...,"facies_logits":...}
      - tuple/list: (ai_pred, facies_logits, ...)
      - single tensor: ai_pred
    """
    ai_pred = None
    facies_logits = None

    if isinstance(out, dict):
        # ai
        for k in ["ai", "ai_pred", "y", "imp", "impedance"]:
            if k in out:
                ai_pred = out[k]
                break
        # facies
        for k in ["facies", "facies_logits", "logits", "facies_logit"]:
            if k in out:
                facies_logits = out[k]
                break

    elif isinstance(out, (list, tuple)):
        if len(out) >= 1:
            ai_pred = out[0]
        if len(out) >= 2:
            facies_logits = out[1]
    else:
        ai_pred = out

    return ai_pred, facies_logits

def mask_center_trace(m_patch):
    """
    m_patch: [1,H,W,T] (dataset 单样本) or [B,1,H,W,T] (batch)
    return mask: [T] or [B,T]
    """
    if m_patch.ndim == 4:
        _, H, W, T = m_patch.shape
        mc = m_patch[0, H//2, W//2, :]
        return mc
    elif m_patch.ndim == 5:
        B, _, H, W, T = m_patch.shape
        mc = m_patch[:, 0, H//2, W//2, :]
        return mc
    else:
        raise ValueError(f"Unexpected m_patch shape: {m_patch.shape}")

def r2_score_np(y, yhat, eps=1e-12):
    # y/yhat: 1D numpy
    y = y.reshape(-1)
    yhat = yhat.reshape(-1)
    sse = np.sum((y - yhat)**2)
    sst = np.sum((y - y.mean())**2)
    return float(1.0 - sse / (sst + eps))


In [7]:
# %%
from collections import defaultdict

@torch.no_grad()
def eval_model_on_test(model, test_idx, name="model"):
    stats = defaultdict(float)
    n_traces = 0

    # facies 汇总（用于 macro-f1）
    fac_y_all = []
    fac_p_all = []

    # 遍历 test_idx（简单直接，和你07一样逐样本）
    for idx in test_idx.tolist():
        b = ds[int(idx)]

        x = b["x"][None].to(device)  # [1,1,H,W,T]
        p = b["p"][None].to(device)  # [1,4,H,W,T]
        c = b["c"][None].to(device)  # [1,1,H,W,T]
        m = b["m"][None].to(device)  # [1,1,H,W,T]

        y = b["y"].to(device).float()              # [T] (norm)
        mc = mask_center_trace(b["m"]).float()     # [T] 0/1
        mask = (mc > 0.5)

        out = model(x, p, c, m)   # ✅完全对齐你07的 forward 调用方式
        ai_pred, facies_logits = unpack_outputs(out)

        # -------- AI回归：保证 shape [T] ----------
        if ai_pred is None:
            raise RuntimeError("Model output has no ai prediction.")

        ai_pred = ai_pred.squeeze()  # [T] 或 [1,T]
        if ai_pred.ndim != 1:
            ai_pred = ai_pred.reshape(-1)

        # 反归一化到真实 AI（更有意义）
        y_dn  = (y * ds.ai_std + ds.ai_mean).detach().cpu().numpy()
        p_dn  = (ai_pred * ds.ai_std + ds.ai_mean).detach().cpu().numpy()
        y_nm  = y.detach().cpu().numpy()
        p_nm  = ai_pred.detach().cpu().numpy()

        if np.any(mask.cpu().numpy()):
            m_np = mask.detach().cpu().numpy().astype(bool)

            # denorm metrics (mask 内)
            yy = y_dn[m_np]
            pp = p_dn[m_np]
            mae = float(np.mean(np.abs(pp - yy)))
            rmse = float(np.sqrt(np.mean((pp - yy)**2)))
            r2 = r2_score_np(yy, pp)

            stats["ai_mae_dn_sum"]  += mae
            stats["ai_rmse_dn_sum"] += rmse
            stats["ai_r2_dn_sum"]   += r2

            # norm metrics (mask 内)
            yy2 = y_nm[m_np]
            pp2 = p_nm[m_np]
            mae2 = float(np.mean(np.abs(pp2 - yy2)))
            rmse2 = float(np.sqrt(np.mean((pp2 - yy2)**2)))
            r22 = r2_score_np(yy2, pp2)

            stats["ai_mae_nm_sum"]  += mae2
            stats["ai_rmse_nm_sum"] += rmse2
            stats["ai_r2_nm_sum"]   += r22

            n_traces += 1

        # -------- facies：只在 valid==1 ----------
        if facies_logits is not None:
            fac_true  = b["facies_center"].long()        # [T] values 0..3 or -1
            fac_valid = b["facies_valid"].float()        # [T] 0/1

            valid = (fac_valid > 0.5) & (fac_true >= 0)
            if valid.any():
                # logits 形状可能是 [C,T] 或 [T,C] 或 [1,C,T]
                L = facies_logits.squeeze()
                if L.ndim == 2:
                    if L.shape[0] == 4:
                        fac_pred = torch.argmax(L, dim=0)   # [T]
                    elif L.shape[1] == 4:
                        fac_pred = torch.argmax(L, dim=1)   # [T]
                    else:
                        fac_pred = None
                elif L.ndim == 1:
                    fac_pred = None
                else:
                    fac_pred = None

                if fac_pred is not None:
                    yt = fac_true[valid].detach().cpu()
                    yp = fac_pred[valid].detach().cpu()
                    fac_y_all.append(yt)
                    fac_p_all.append(yp)

    # 汇总
    out = {}
    out["name"] = name
    out["n_traces"] = n_traces

    if n_traces > 0:
        out["AI_MAE_denorm"]  = stats["ai_mae_dn_sum"]  / n_traces
        out["AI_RMSE_denorm"] = stats["ai_rmse_dn_sum"] / n_traces
        out["AI_R2_denorm"]   = stats["ai_r2_dn_sum"]   / n_traces

        out["AI_MAE_norm"]  = stats["ai_mae_nm_sum"]  / n_traces
        out["AI_RMSE_norm"] = stats["ai_rmse_nm_sum"] / n_traces
        out["AI_R2_norm"]   = stats["ai_r2_nm_sum"]   / n_traces

    # facies macro-f1
    if len(fac_y_all) > 0:
        yt = torch.cat(fac_y_all).numpy().astype(np.int64)
        yp = torch.cat(fac_p_all).numpy().astype(np.int64)

        acc = float(np.mean(yt == yp))
        out["Facies_Acc"] = acc

        # per-class f1 + macro f1
        f1s = []
        for c in range(4):
            tp = np.sum((yt == c) & (yp == c))
            fp = np.sum((yt != c) & (yp == c))
            fn = np.sum((yt == c) & (yp != c))
            denom = (2*tp + fp + fn)
            f1 = (2*tp/denom) if denom > 0 else 0.0
            f1s.append(float(f1))
        out["Facies_F1_per_class"] = f1s
        out["Facies_MacroF1"] = float(np.mean(f1s))
        out["Facies_n_valid"] = int(len(yt))
    else:
        out["Facies_Acc"] = None
        out["Facies_MacroF1"] = None
        out["Facies_n_valid"] = 0

    return out

In [8]:
# %%
test_idx_np = test_idx  # numpy array from splits
print("test size:", len(test_idx_np))

res_joint = eval_model_on_test(m_joint, test_idx_np, name="best_joint")
res_ai    = eval_model_on_test(m_ai,    test_idx_np, name="best_ai")

res_joint, res_ai

test size: 45


({'name': 'best_joint',
  'n_traces': 45,
  'AI_MAE_denorm': 0.5437761776977115,
  'AI_RMSE_denorm': 0.7187682429949442,
  'AI_R2_denorm': 0.6442147281434801,
  'AI_MAE_norm': 0.43094752033551537,
  'AI_RMSE_norm': 0.5696303049723307,
  'AI_R2_norm': 0.6442147374153138,
  'Facies_Acc': 0.8294807370184255,
  'Facies_F1_per_class': [0.9254853944376421,
   0.1711229946524064,
   0.7651205936920222,
   0.0650887573964497],
  'Facies_MacroF1': 0.4817044350446301,
  'Facies_n_valid': 8955},
 {'name': 'best_ai',
  'n_traces': 45,
  'AI_MAE_denorm': 0.5366928882069057,
  'AI_RMSE_denorm': 0.7127246002356211,
  'AI_R2_denorm': 0.6501044604513381,
  'AI_MAE_norm': 0.4253339535660214,
  'AI_RMSE_norm': 0.5648406657907697,
  'AI_R2_norm': 0.650104464424981,
  'Facies_Acc': 0.8219988833054159,
  'Facies_F1_per_class': [0.9243507059438731,
   0.0,
   0.7541284403669725,
   0.018404907975460124],
  'Facies_MacroF1': 0.4242210135715764,
  'Facies_n_valid': 8955})

In [9]:
# %%
out_dir = os.path.join(paths.processed_dir, "eval_reports")
os.makedirs(out_dir, exist_ok=True)

report = {
    "data_root": DATA_ROOT,
    "processed_dir": paths.processed_dir,
    "constraints_npz": constraints_npz,
    "ckpt_dir": ckpt_dir_final,
    "ckpt_best_joint": ckpt_joint,
    "ckpt_best_ai": ckpt_ai,
    "results": {
        "best_joint": res_joint,
        "best_ai": res_ai
    }
}

out_path = os.path.join(out_dir, "eval_multitask_test_final.json")
with open(out_path, "w", encoding="utf-8") as f:
    json.dump(report, f, indent=2, ensure_ascii=False)

print("Saved:", out_path)
print(json.dumps(report["results"], indent=2, ensure_ascii=False))

Saved: H:\GK-MRL-PhysicsConsistent-Inversion\GK-MRL-PhysicsConsistent-Inversion\data\processed\eval_reports\eval_multitask_test_final.json
{
  "best_joint": {
    "name": "best_joint",
    "n_traces": 45,
    "AI_MAE_denorm": 0.5437761776977115,
    "AI_RMSE_denorm": 0.7187682429949442,
    "AI_R2_denorm": 0.6442147281434801,
    "AI_MAE_norm": 0.43094752033551537,
    "AI_RMSE_norm": 0.5696303049723307,
    "AI_R2_norm": 0.6442147374153138,
    "Facies_Acc": 0.8294807370184255,
    "Facies_F1_per_class": [
      0.9254853944376421,
      0.1711229946524064,
      0.7651205936920222,
      0.0650887573964497
    ],
    "Facies_MacroF1": 0.4817044350446301,
    "Facies_n_valid": 8955
  },
  "best_ai": {
    "name": "best_ai",
    "n_traces": 45,
    "AI_MAE_denorm": 0.5366928882069057,
    "AI_RMSE_denorm": 0.7127246002356211,
    "AI_R2_denorm": 0.6501044604513381,
    "AI_MAE_norm": 0.4253339535660214,
    "AI_RMSE_norm": 0.5648406657907697,
    "AI_R2_norm": 0.650104464424981,
    "F

In [10]:
# %%
import os
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix

In [11]:
# %%
# 输出目录
FIG_DIR = os.path.join(paths.processed_dir, "eval_reports", "figs")
os.makedirs(FIG_DIR, exist_ok=True)
print("FIG_DIR:", FIG_DIR)

# physics forward（可选）
try:
    from src.physics_forward import forward_seismic_from_ai
    HAS_PHYS = True
    print("Physics forward: OK")
except Exception as e:
    HAS_PHYS = False
    print("Physics forward: NOT available -> skip physics figs. Error:", repr(e))

def r2_score_np(y, yhat, eps=1e-12):
    y = y.reshape(-1)
    yhat = yhat.reshape(-1)
    sse = np.sum((y - yhat)**2)
    sst = np.sum((y - y.mean())**2)
    return float(1.0 - sse / (sst + eps))

@torch.no_grad()
def predict_one(model, idx):
    """
    返回 dict:
      wellname, il, xl,
      y_norm[T], yhat_norm[T],
      y_denorm[T], yhat_denorm[T],
      mask[T],
      fac_true[T], fac_valid[T], fac_pred[T] or None,
      seis_obs_denorm[T], seis_syn_denorm[T] or None
    """
    b = ds[int(idx)]
    x = b["x"][None].to(device)
    p = b["p"][None].to(device)
    c = b["c"][None].to(device)
    m = b["m"][None].to(device)

    out = model(x, p, c, m)
    ai_pred, facies_logits = unpack_outputs(out)

    # AI pred -> [T]
    ai_pred = ai_pred.squeeze()
    if ai_pred.ndim != 1:
        ai_pred = ai_pred.reshape(-1)

    y = b["y"].float().to(device)              # [T] norm
    mask = mask_center_trace(b["m"]).float()   # [T] 0/1
    mask_np = (mask.cpu().numpy() > 0.5)

    # denorm
    y_den  = (y * ds.ai_std + ds.ai_mean).detach().cpu().numpy()
    p_den  = (ai_pred * ds.ai_std + ds.ai_mean).detach().cpu().numpy()
    y_norm = y.detach().cpu().numpy()
    p_norm = ai_pred.detach().cpu().numpy()

    # facies
    fac_true  = b["facies_center"].long().cpu().numpy()      # [T] in {0..3} or -1
    fac_valid = b["facies_valid"].float().cpu().numpy()      # [T] 0/1
    fac_pred = None
    if facies_logits is not None:
        L = facies_logits.squeeze().detach().cpu()
        if L.ndim == 2:
            if L.shape[0] == 4:   # [4,T]
                fac_pred = torch.argmax(L, dim=0).numpy()
            elif L.shape[1] == 4: # [T,4]
                fac_pred = torch.argmax(L, dim=1).numpy()

    # physics: observed seismic center trace (denorm)
    x_patch = b["x"].float()  # [1,H,W,T] norm
    H = x_patch.shape[1]; W = x_patch.shape[2]
    seis_obs_norm = x_patch[0, H//2, W//2, :].numpy()
    seis_obs_den  = seis_obs_norm * float(ds.seis_std) + float(ds.seis_mean)

    seis_syn_den = None
    if HAS_PHYS:
        try:
            ai_den_t = torch.from_numpy(p_den).to(device).float()[None, :]  # [1,T]
            seis_syn = forward_seismic_from_ai(ai_den_t)
            seis_syn = seis_syn.squeeze().detach().cpu().numpy()
            if seis_syn.ndim != 1:
                seis_syn = seis_syn.reshape(-1)
            seis_syn_den = seis_syn
        except Exception:
            seis_syn_den = None

    return dict(
        wellname=b["wellname"],
        il=int(b["il"]),
        xl=int(b["xl"]),
        y_norm=y_norm, yhat_norm=p_norm,
        y_den=y_den,   yhat_den=p_den,
        mask=mask_np,
        fac_true=fac_true,
        fac_valid=fac_valid,
        fac_pred=fac_pred,
        seis_obs_den=seis_obs_den,
        seis_syn_den=seis_syn_den,
    )

def metrics_ai_mask(y_den, yhat_den, mask):
    if mask.sum() == 0:
        return dict(mae=np.nan, rmse=np.nan, r2=np.nan)
    yy = y_den[mask]
    pp = yhat_den[mask]
    mae = float(np.mean(np.abs(pp - yy)))
    rmse = float(np.sqrt(np.mean((pp - yy)**2)))
    r2 = r2_score_np(yy, pp)
    return dict(mae=mae, rmse=rmse, r2=r2)

def metrics_phys(seis_obs_den, seis_syn_den, mask):
    if (seis_syn_den is None) or (mask.sum() == 0):
        return dict(phys_mse=np.nan)
    yy = seis_obs_den[mask]
    pp = seis_syn_den[mask]
    mse = float(np.mean((pp - yy)**2))
    return dict(phys_mse=mse)

FIG_DIR: H:\GK-MRL-PhysicsConsistent-Inversion\GK-MRL-PhysicsConsistent-Inversion\data\processed\eval_reports\figs
Physics forward: OK


In [12]:
# %%
rows = []
for idx in test_idx.tolist():
    a = predict_one(m_ai, idx)
    j = predict_one(m_joint, idx)

    ai_m = metrics_ai_mask(a["y_den"], a["yhat_den"], a["mask"])
    jo_m = metrics_ai_mask(j["y_den"], j["yhat_den"], j["mask"])

    # physics
    ai_p = metrics_phys(a["seis_obs_den"], a["seis_syn_den"], a["mask"])
    jo_p = metrics_phys(j["seis_obs_den"], j["seis_syn_den"], j["mask"])

    rows.append({
        "wellname": a["wellname"],
        "il": a["il"], "xl": a["xl"],
        "AI_MAE_best_ai": ai_m["mae"],
        "AI_RMSE_best_ai": ai_m["rmse"],
        "AI_R2_best_ai": ai_m["r2"],
        "AI_MAE_best_joint": jo_m["mae"],
        "AI_RMSE_best_joint": jo_m["rmse"],
        "AI_R2_best_joint": jo_m["r2"],
        "Delta_MAE(joint-ai)": jo_m["mae"] - ai_m["mae"],   # <0 表示 joint 更好
        "Delta_RMSE(joint-ai)": jo_m["rmse"] - ai_m["rmse"],
        "Delta_R2(joint-ai)": jo_m["r2"] - ai_m["r2"],      # >0 表示 joint 更好
        "Phys_MSE_best_ai": ai_p["phys_mse"],
        "Phys_MSE_best_joint": jo_p["phys_mse"],
        "Delta_PhysMSE(joint-ai)": jo_p["phys_mse"] - ai_p["phys_mse"],
    })

df_well = pd.DataFrame(rows)

# 排序：最“提升”的井（joint 相对 ai：MAE 降得最多）
df_improve = df_well.sort_values("Delta_MAE(joint-ai)", ascending=True).reset_index(drop=True)
df_worsen  = df_well.sort_values("Delta_MAE(joint-ai)", ascending=False).reset_index(drop=True)

out_csv = os.path.join(FIG_DIR, "per_well_compare_best_joint_vs_best_ai.csv")
df_well.to_csv(out_csv, index=False, encoding="utf-8-sig")
print("Saved:", out_csv)

print("Top improved (joint better):")
display(df_improve.head(10))

print("Top worsened (ai better):")
display(df_worsen.head(10))

Saved: H:\GK-MRL-PhysicsConsistent-Inversion\GK-MRL-PhysicsConsistent-Inversion\data\processed\eval_reports\figs\per_well_compare_best_joint_vs_best_ai.csv
Top improved (joint better):


,wellname,il,xl,AI_MAE_best_ai,AI_RMSE_best_ai,AI_R2_best_ai,AI_MAE_best_joint,AI_RMSE_best_joint,AI_R2_best_joint,Delta_MAE(joint-ai),Delta_RMSE(joint-ai),Delta_R2(joint-ai),Phys_MSE_best_ai,Phys_MSE_best_joint,Delta_PhysMSE(joint-ai)
0,VW0148,76,77,0.545120,0.713699,0.671174,0.544828,0.717391,0.667763,-0.000291,0.003692,-0.003411,46.689053,46.687710,-0.001343
1,VW0150,76,96,0.667726,0.851721,0.556999,0.668229,0.852389,0.556304,0.000503,0.000668,-0.000695,45.587608,45.587135,-0.000473
2,VW0235,112,143,0.575087,0.768908,0.573458,0.578245,0.773561,0.568280,0.003158,0.004653,-0.005178,55.330841,55.330021,-0.000820
3,VW0060,29,190,0.557627,0.761716,0.585697,0.560801,0.763861,0.583360,0.003174,0.002145,-0.002337,53.193802,53.193127,-0.000675
4,VW0263,131,30,0.357512,0.502218,0.796858,0.361070,0.505174,0.794460,0.003558,0.002956,-0.002398,53.713261,53.712570,-0.000690
5,VW0053,29,124,0.640584,0.812523,0.548437,0.644329,0.815549,0.545067,0.003745,0.003026,-0.003369,52.129494,52.128876,-0.000618
6,VW0175,85,143,0.584198,0.772015,0.653727,0.588585,0.775849,0.650280,0.004387,0.003833,-0.003447,56.160961,56.161278,0.000317
7,VW0108,57,77,0.695736,0.896812,0.528375,0.700508,0.903066,0.521774,0.004772,0.006254,-0.006601,42.201378,42.197750,-0.003628
8,VW0189,94,86,0.490923,0.663988,0.711386,0.495748,0.668695,0.707279,0.004825,0.004707,-0.004106,49.168808,49.166870,-0.001938
9,VW0013,11,124,0.517665,0.714912,0.576758,0.522534,0.718525,0.572469,0.004869,0.003613,-0.004289,52.445389,52.443077,-0.002312


Top worsened (ai better):


,wellname,il,xl,AI_MAE_best_ai,AI_RMSE_best_ai,AI_R2_best_ai,AI_MAE_best_joint,AI_RMSE_best_joint,AI_R2_best_joint,Delta_MAE(joint-ai),Delta_RMSE(joint-ai),Delta_R2(joint-ai),Phys_MSE_best_ai,Phys_MSE_best_joint,Delta_PhysMSE(joint-ai)
0,VW0046,29,58,0.483917,0.662753,0.756480,0.498039,0.671662,0.749889,0.014122,0.008909,-0.006591,51.108818,51.108231,-0.000587
1,VW0115,57,143,0.661761,0.850493,0.590009,0.674391,0.860501,0.580303,0.012630,0.010008,-0.009706,53.645889,53.642990,-0.002899
2,VW0002,11,20,0.381351,0.567602,0.709960,0.392551,0.571710,0.705747,0.011200,0.004107,-0.004213,62.343781,62.342930,-0.000851
3,VW0205,103,49,0.452665,0.582403,0.815218,0.463519,0.590882,0.809798,0.010854,0.008479,-0.005420,50.870094,50.871025,0.000931
4,VW0047,29,68,0.485971,0.647306,0.718225,0.496758,0.653850,0.712499,0.010787,0.006544,-0.005726,48.477196,48.473190,-0.004005
5,VW0204,103,39,0.551888,0.719345,0.735310,0.562262,0.728241,0.728723,0.010374,0.008896,-0.006587,52.493614,52.492741,-0.000874
6,VW0281,140,11,0.380308,0.505086,0.785724,0.390631,0.515167,0.777085,0.010322,0.010081,-0.008638,57.847050,57.844784,-0.002266
7,VW0297,140,162,0.402256,0.580931,0.575239,0.412562,0.588611,0.563934,0.010305,0.007680,-0.011305,60.430843,60.429096,-0.001747
8,VW0040,20,190,0.586250,0.749431,0.584264,0.596004,0.757170,0.575633,0.009755,0.007739,-0.008630,52.194004,52.196953,0.002949
9,VW0231,112,105,0.559858,0.744604,0.593868,0.569357,0.753257,0.584374,0.009500,0.008653,-0.009494,47.895912,47.891640,-0.004272


In [13]:
# %%
def plot_ai_compare(idx, tag=""):
    a = predict_one(m_ai, idx)
    j = predict_one(m_joint, idx)

    t = np.arange(len(a["y_den"]))
    mask = a["mask"]

    plt.figure()
    plt.plot(t, a["y_den"], label="True AI (denorm)", linewidth=1.6)
    plt.plot(t, a["yhat_den"], label="best_ai", alpha=0.9)
    plt.plot(t, j["yhat_den"], label="best_joint", alpha=0.9)

    # mask 作为背景（不指定颜色，使用默认）
    yymin = np.nanmin(a["y_den"])
    yymax = np.nanmax(a["y_den"])
    mask_band = np.where(mask, yymax, np.nan)
    plt.plot(t, mask_band, label="Mask(top)", alpha=0.25)

    plt.title(f"{a['wellname']}  IL={a['il']} XL={a['xl']}")
    plt.xlabel("TWT (ms)")
    plt.ylabel("Acoustic Impedance")
    plt.grid(True)
    plt.legend(loc="best")

    fn = f"AI_compare_{tag}_{a['wellname']}_IL{a['il']}_XL{a['xl']}.png"
    fp = os.path.join(FIG_DIR, fn)
    plt.tight_layout()
    plt.savefig(fp, dpi=200)
    plt.close()
    return fp

# 画：提升最多的 6 口井 + 变差最多的 6 口井
topN = min(6, len(df_improve))
botN = min(6, len(df_worsen))

saved = []
for wn in df_improve.head(topN)["wellname"].tolist():
    idx = int(np.where([s.wellname == wn for s in ds.samples])[0][0])
    saved.append(plot_ai_compare(idx, tag="improve"))

for wn in df_worsen.head(botN)["wellname"].tolist():
    idx = int(np.where([s.wellname == wn for s in ds.samples])[0][0])
    saved.append(plot_ai_compare(idx, tag="worsen"))

print("Saved AI compare figs:")
for p in saved:
    print(" ", p)

Saved AI compare figs:
  H:\GK-MRL-PhysicsConsistent-Inversion\GK-MRL-PhysicsConsistent-Inversion\data\processed\eval_reports\figs\AI_compare_improve_VW0148_IL76_XL77.png
  H:\GK-MRL-PhysicsConsistent-Inversion\GK-MRL-PhysicsConsistent-Inversion\data\processed\eval_reports\figs\AI_compare_improve_VW0150_IL76_XL96.png
  H:\GK-MRL-PhysicsConsistent-Inversion\GK-MRL-PhysicsConsistent-Inversion\data\processed\eval_reports\figs\AI_compare_improve_VW0235_IL112_XL143.png
  H:\GK-MRL-PhysicsConsistent-Inversion\GK-MRL-PhysicsConsistent-Inversion\data\processed\eval_reports\figs\AI_compare_improve_VW0060_IL29_XL190.png
  H:\GK-MRL-PhysicsConsistent-Inversion\GK-MRL-PhysicsConsistent-Inversion\data\processed\eval_reports\figs\AI_compare_improve_VW0263_IL131_XL30.png
  H:\GK-MRL-PhysicsConsistent-Inversion\GK-MRL-PhysicsConsistent-Inversion\data\processed\eval_reports\figs\AI_compare_improve_VW0053_IL29_XL124.png
  H:\GK-MRL-PhysicsConsistent-Inversion\GK-MRL-PhysicsConsistent-Inversion\data\proc

In [14]:
# %%
def collect_facies_for_model(model):
    yt_all, yp_all = [], []
    for idx in test_idx.tolist():
        r = predict_one(model, idx)
        if r["fac_pred"] is None:
            continue
        valid = (r["fac_valid"] > 0.5) & (r["fac_true"] >= 0)
        if valid.sum() == 0:
            continue
        yt_all.append(r["fac_true"][valid])
        yp_all.append(r["fac_pred"][valid])
    if len(yt_all) == 0:
        return None, None
    return np.concatenate(yt_all), np.concatenate(yp_all)

yt_j, yp_j = collect_facies_for_model(m_joint)
yt_a, yp_a = collect_facies_for_model(m_ai)

def plot_cm(yt, yp, title, fname):
    cm = confusion_matrix(yt, yp, labels=[0,1,2,3])
    plt.figure()
    plt.imshow(cm)  # 不指定颜色
    plt.title(title)
    plt.xlabel("Pred")
    plt.ylabel("True")
    plt.xticks([0,1,2,3], ["0","1","2","3"])
    plt.yticks([0,1,2,3], ["0","1","2","3"])
    # 标注数值
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, str(cm[i,j]), ha="center", va="center")
    plt.tight_layout()
    fp = os.path.join(FIG_DIR, fname)
    plt.savefig(fp, dpi=200)
    plt.close()
    return fp, cm

if yt_j is None:
    print("No facies logits found in model outputs -> skip confusion matrix.")
else:
    fpj, cmj = plot_cm(yt_j, yp_j, "Facies Confusion Matrix (best_joint, valid only)", "cm_best_joint.png")
    fpa, cma = plot_cm(yt_a, yp_a, "Facies Confusion Matrix (best_ai, valid only)", "cm_best_ai.png")
    print("Saved:", fpj)
    print("Saved:", fpa)

Saved: H:\GK-MRL-PhysicsConsistent-Inversion\GK-MRL-PhysicsConsistent-Inversion\data\processed\eval_reports\figs\cm_best_joint.png
Saved: H:\GK-MRL-PhysicsConsistent-Inversion\GK-MRL-PhysicsConsistent-Inversion\data\processed\eval_reports\figs\cm_best_ai.png


In [15]:
# %%
if not HAS_PHYS:
    print("Physics forward not available -> skip physics plots.")
else:
    # 取物理误差列（mask 内 MSE）
    v1 = df_well["Phys_MSE_best_joint"].to_numpy()
    v2 = df_well["Phys_MSE_best_ai"].to_numpy()

    # 1) 直方图对比（不指定颜色）
    plt.figure()
    plt.hist(v2[~np.isnan(v2)], bins=30, alpha=0.6, label="best_ai")
    plt.hist(v1[~np.isnan(v1)], bins=30, alpha=0.6, label="best_joint")
    plt.title("Physics Consistency (MSE, mask only)")
    plt.xlabel("MSE")
    plt.ylabel("Count")
    plt.grid(True)
    plt.legend()
    fp = os.path.join(FIG_DIR, "physics_mse_hist.png")
    plt.tight_layout()
    plt.savefig(fp, dpi=200)
    plt.close()
    print("Saved:", fp)

    # 2) 散点：AI MAE vs Phys MSE（joint）
    plt.figure()
    x = df_well["AI_MAE_best_joint"].to_numpy()
    y = df_well["Phys_MSE_best_joint"].to_numpy()
    ok = (~np.isnan(x)) & (~np.isnan(y))
    plt.scatter(x[ok], y[ok], s=12)
    plt.title("best_joint: AI MAE vs Physics MSE (mask only)")
    plt.xlabel("AI MAE")
    plt.ylabel("Physics MSE")
    plt.grid(True)
    fp = os.path.join(FIG_DIR, "scatter_joint_aiMAE_vs_physMSE.png")
    plt.tight_layout()
    plt.savefig(fp, dpi=200)
    plt.close()
    print("Saved:", fp)

    # 3) joint vs ai：ΔPhysMSE 分布
    plt.figure()
    d = df_well["Delta_PhysMSE(joint-ai)"].to_numpy()
    plt.hist(d[~np.isnan(d)], bins=30)
    plt.title("Delta Physics MSE (best_joint - best_ai)")
    plt.xlabel("ΔMSE (negative = joint better)")
    plt.ylabel("Count")
    plt.grid(True)
    fp = os.path.join(FIG_DIR, "physics_delta_mse_hist.png")
    plt.tight_layout()
    plt.savefig(fp, dpi=200)
    plt.close()
    print("Saved:", fp)

Saved: H:\GK-MRL-PhysicsConsistent-Inversion\GK-MRL-PhysicsConsistent-Inversion\data\processed\eval_reports\figs\physics_mse_hist.png
Saved: H:\GK-MRL-PhysicsConsistent-Inversion\GK-MRL-PhysicsConsistent-Inversion\data\processed\eval_reports\figs\scatter_joint_aiMAE_vs_physMSE.png
Saved: H:\GK-MRL-PhysicsConsistent-Inversion\GK-MRL-PhysicsConsistent-Inversion\data\processed\eval_reports\figs\physics_delta_mse_hist.png
